In [ ]:
#Text similarity app	Cosine similarity, semantic matching	Quora questions

In [ ]:
#USED THIS CODE TO UNZIP THE TEST AND SUBMISSION .CSV FILES AND AFTER EXTRACTION I DELETED THE ZIP FILES FROM DIRECTORY SO AFTER DELETION THIS CODE DOESNT WORKS
# import zipfile
# import os
# project_dir=r"C:\Users\LENOVO\Desktop\NN_projects\level3ML\textsimilarityApp\data\quora-question-pairs"
# zip_files=["train.csv.zip","sample_submission.csv.zip"]
# for zip_name in zip_files:
#     zip_path=os.path.join(project_dir,zip_name)
#     with zipfile.ZipFile(zip_path,"r") as zip_ref:
#         zip_ref.extractall(project_dir)#Extracting zip files inside the project directory
#         print(f"{zip_name} unzipped successfully .")

In [ ]:
import pandas as pd
import numpy as np
import torch

In [ ]:
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU device: {torch.cuda.get_device_name(0)}")
    print(f"GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

In [ ]:
train_df = pd.read_csv('train.csv')  
test_df = pd.read_csv('test.csv')
submission_df = pd.read_csv('sample_submission.csv')

print(train_df.head())
print(test_df.head())
print(submission_df.head())


In [ ]:
import re
import string

In [ ]:
def clean_text(text):
    if (pd.isna(text)):
        return ""
    text=text.lower()#converting all the text to lowercase
    text=re.sub(r'[^a-zA-Z0-9\s]'," ",text)#Rempoving all special characters
    text=re.sub(r'\s+'," ",text).strip()#removing extra spaces
    return text
train_df['question1']=train_df['question1'].apply(clean_text)
train_df['question2']=train_df['question2'].apply(clean_text)
test_df['question1']=test_df['question1'].apply(clean_text)
test_df['question2']=test_df['question2'].apply(clean_text)
print(train_df.head())
print(train_df['question1'][0]) 


In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stopwords=set(stopwords.words('english'))
print(stopwords)
      

In [ ]:
def remove_stopwords(text):
    tokens=word_tokenize(text)
    filtered_tokens=[word for word in tokens if word not in stopwords]
    filtered_text=" ".join(filtered_tokens)
    return filtered_text
train_df['question1']=train_df['question1'].apply(remove_stopwords)
train_df['question2']=train_df['question2'].apply(remove_stopwords)
test_df['question1']=test_df['question1'].apply(remove_stopwords)
test_df['question2']=test_df['question2'].apply(remove_stopwords)
print(train_df['question1'][0])


In [ ]:
nltk.download('punkt_tab')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger_eng')
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()



In [ ]:
def reduce_to_base(text):
    tokens=word_tokenize(text)
    lemmatized_tokens=[lemmatizer.lemmatize(words) for words in tokens]
    lemmatized_text=" ".join(lemmatized_tokens)
    return lemmatized_text
train_df['question1']=train_df['question1'].apply(reduce_to_base)
train_df['question2']=train_df['question2'].apply(reduce_to_base)
test_df['question1']=test_df['question1'].apply(reduce_to_base)
test_df['question2']=test_df['question2'].apply(reduce_to_base)

In [ ]:
import sys
print(sys.executable)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from sklearn.preprocessing import normalize

In [ ]:
def compute_simple_features(df):
    #Compute length differrence and common word features
    df['lenq1']=df['question1'].apply(lambda x:len(x.split()))#no of words in q1
    df['lenq2']=df['question2'].apply(lambda x:len(x.split()))#no of words in q2
    df['len_diff']=abs(df['lenq1']-df['lenq2'])#Absolute length difference
    #no of common words
    df['common_words']=df.apply(lambda row :len(set(row['question1'].split()).intersection(set(row['question2'].split()))),axis=1)
    return df
train_df=compute_simple_features(train_df)
test_df=compute_simple_features(test_df)

In [ ]:
# def compute_tfidf_cosine_similarity(train_df,test_df):
#     #compute cosine similarity of tfIDf vectors of question pairs
#     #Combine all questions to fit the TF-IDF vectorizer
#     all_questions=pd.concat([train_df['question1'],train_df['question2'],test_df['question1'],test_df['question2']])
#     tfidf_vectorizer=TfidfVectorizer()
#     tfidf_vectorizer.fit(all_questions)
#     #Transform questions to TF-IDF vectors
#     train_q1_tfidf=tfidf_vectorizer.transform(train_df['question1'])
#     train_q2_tfidf=tfidf_vectorizer.transform(train_df['question2'])
#     test_q1_tfidf=tfidf_vectorizer.transform(test_df['question1'])
#     test_q2_tfidf=tfidf_vectorizer.transform(test_df['question2'])
#     #Compute cosine similarity
#     train_df['tfidf_cosine_sim']=[cosine_similarity(train_q1_tfidf[i],train_q2_tfidf[i])[0][0] for i in range(len(train_df))]
#     test_df['tfidf_cosine_sim']=[cosine_similarity(test_q1_tfidf[i],test_q2_tfidf[i])[0][0] for i in range(len(test_df))]
#     return train_df,test_df
# train_df,test_df=compute_tfidf_cosine_similarity(train_df,test_df)


In [ ]:
def compute_tfidf_cosine_similarity(train_df, test_df, max_features=20000):
    # Combine all questions to fit the TF-IDF vectorizer
    all_questions = pd.concat([
        train_df['question1'], train_df['question2'],
        test_df['question1'], test_df['question2']
    ])
    
    tfidf_vectorizer = TfidfVectorizer(max_features=max_features)
    tfidf_vectorizer.fit(all_questions)
    
    # Transform train & test questions
    train_q1_tfidf = tfidf_vectorizer.transform(train_df['question1'])
    train_q2_tfidf = tfidf_vectorizer.transform(train_df['question2'])
    test_q1_tfidf = tfidf_vectorizer.transform(test_df['question1'])
    test_q2_tfidf = tfidf_vectorizer.transform(test_df['question2'])
    
    # Normalize rows (L2 norm = 1), so dot product = cosine similarity
    train_q1_norm = normalize(train_q1_tfidf)
    train_q2_norm = normalize(train_q2_tfidf)
    test_q1_norm = normalize(test_q1_tfidf)
    test_q2_norm = normalize(test_q2_tfidf)
    
    # Compute cosine similarity as row-wise dot product (fast!)
    train_df['tfidf_cosine_sim'] = (train_q1_norm.multiply(train_q2_norm)).sum(axis=1).A1
    test_df['tfidf_cosine_sim'] = (test_q1_norm.multiply(test_q2_norm)).sum(axis=1).A1
    
    return train_df, test_df
train_df,test_df=compute_tfidf_cosine_similarity(train_df,test_df)


In [ ]:
# def compute_embedding_similarity(train_df, test_df,batch_size=128):
#     #initialize model with gpu support
#     device = 'cuda' if torch.cuda.is_available() else 'cpu'
#     model = SentenceTransformer('all-MiniLM-L6-v2', device=device)
    
#     print(f"Using device: {device}")

#     print("Computing embeddings for training data...")
#     train_q1_embeddings = model.encode(train_df['question1'].tolist(),batch_size=batch_size, show_progress_bar=True, convert_to_tensor=True,device=device)
#     train_q2_embeddings = model.encode(train_df['question2'].tolist(),batch_size=batch_size, show_progress_bar=True, convert_to_tensor=True,device=device)

#     print("Computing embeddings for test data...")
#     test_q1_embeddings = model.encode(test_df['question1'].tolist(),batch_size=batch_size, show_progress_bar=True, convert_to_tensor=True,device=device)
#     test_q2_embeddings = model.encode(test_df['question2'].tolist(),batch_size=batch_size, show_progress_bar=True, convert_to_tensor=True,device=device)

#     #  Vectorized cosine similarity 
#     # Normalize embeddings 
#     train_q1_norm = torch.nn.functional.normalize(train_q1_embeddings, dim=1)
#     train_q2_norm = torch.nn.functional.normalize(train_q2_embeddings, dim=1)
#     test_q1_norm = torch.nn.functional.normalize(test_q1_embeddings, dim=1)
#     test_q2_norm = torch.nn.functional.normalize(test_q2_embeddings, dim=1)    

#     # Cosine similarity = dot product after normalization
#     train_sim = torch.sum(train_q1_norm * train_q2_norm, dim=1)
#     test_sim  = torch.sum(test_q1_norm * test_q2_norm, dim=1)

#     # Store results
#     train_df['embedding_cosine_similarity'] = train_sim.cpu().numpy()
#     test_df['embedding_cosine_similarity'] = test_sim.cpu().numpy()

#         # Clear GPU memory
#     if torch.cuda.is_available():
#         torch.cuda.empty_cache()

#     return train_df, test_df
# train_df, test_df = compute_embedding_similarity(train_df, test_df,batch_size=128)


In [ ]:
def compute_embedding_similarity(train_df, test_df, batch_size=32, chunk_size=10000):
    """
    Memory-optimized GPU embedding computation with chunking and error handling
    """
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    
    try:
        model = SentenceTransformer('all-MiniLM-L6-v2', device=device)
        print(f"Using device: {device}")
        
        # Clear GPU cache before starting
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
            
        # Function to process data in chunks
        def process_in_chunks(texts, description="Processing", current_batch_size=batch_size):
            embeddings_list = []
            
            for i in range(0, len(texts), chunk_size):
                chunk_texts = texts[i:i+chunk_size]
                chunk_num = i//chunk_size + 1
                total_chunks = (len(texts)-1)//chunk_size + 1
                print(f"{description} chunk {chunk_num}/{total_chunks} (size: {len(chunk_texts)})")
                
                try:
                    chunk_embeddings = model.encode(
                        chunk_texts,
                        batch_size=current_batch_size,
                        show_progress_bar=True,
                        convert_to_tensor=True,
                        device=device
                    )
                    embeddings_list.append(chunk_embeddings)
                    
                    # Clear cache after each chunk
                    if torch.cuda.is_available():
                        torch.cuda.empty_cache()
                        
                except RuntimeError as e:
                    if "out of memory" in str(e).lower():
                        print(f"GPU OOM error. Reducing batch size from {current_batch_size} to {current_batch_size//2}")
                        current_batch_size = max(1, current_batch_size//2)
                        
                        # Clear cache and retry
                        if torch.cuda.is_available():
                            torch.cuda.empty_cache()
                            
                        # Retry with smaller batch
                        chunk_embeddings = model.encode(
                            chunk_texts,
                            batch_size=current_batch_size,
                            show_progress_bar=True,
                            convert_to_tensor=True,
                            device=device
                        )
                        embeddings_list.append(chunk_embeddings)
                    else:
                        raise e
                        
            # Concatenate all chunks
            return torch.cat(embeddings_list, dim=0)
        
        # Process training data in chunks
        print("Computing embeddings for training questions...")
        train_q1_embeddings = process_in_chunks(
            train_df['question1'].tolist(),
            "Training Q1"
        )
        train_q2_embeddings = process_in_chunks(
            train_df['question2'].tolist(), 
            "Training Q2"
        )
        
        # Process test data in chunks  
        print("Computing embeddings for test questions...")
        test_q1_embeddings = process_in_chunks(
            test_df['question1'].tolist(),
            "Test Q1"
        )
        test_q2_embeddings = process_in_chunks(
            test_df['question2'].tolist(),
            "Test Q2"
        )
        
        print("Computing cosine similarities...")
        
        # Process similarity computation in chunks to avoid memory issues
        def compute_similarity_chunks(emb1, emb2, sim_chunk_size=50000):
            similarities = []
            
            for i in range(0, len(emb1), sim_chunk_size):
                end_idx = min(i + sim_chunk_size, len(emb1))
                print(f"Computing similarity chunk {i//sim_chunk_size + 1}/{(len(emb1)-1)//sim_chunk_size + 1}")
                
                # Normalize embeddings
                chunk_emb1 = torch.nn.functional.normalize(emb1[i:end_idx], dim=1)
                chunk_emb2 = torch.nn.functional.normalize(emb2[i:end_idx], dim=1)
                
                # Compute similarity
                chunk_sim = torch.sum(chunk_emb1 * chunk_emb2, dim=1)
                similarities.append(chunk_sim.cpu().numpy())
                
                # Clear cache
                if torch.cuda.is_available():
                    torch.cuda.empty_cache()
                    
            return np.concatenate(similarities)
        
        # Compute similarities in chunks
        train_similarities = compute_similarity_chunks(train_q1_embeddings, train_q2_embeddings)
        test_similarities = compute_similarity_chunks(test_q1_embeddings, test_q2_embeddings)
        
        # Store results
        train_df['embedding_cosine_similarity'] = train_similarities
        test_df['embedding_cosine_similarity'] = test_similarities
        
        # Final cleanup
        del train_q1_embeddings, train_q2_embeddings, test_q1_embeddings, test_q2_embeddings
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
            
        print("GPU embedding computation completed successfully!")
        return train_df, test_df
        
    except Exception as e:
        print(f"GPU processing failed with error: {e}")
        print("Falling back to CPU processing...")
        
        # Fallback to CPU
        return compute_embedding_similarity_cpu_fallback(train_df, test_df)


def compute_embedding_similarity_cpu_fallback(train_df, test_df):
    """
    CPU fallback function in case GPU fails
    """
    print("Using CPU fallback...")
    model = SentenceTransformer('all-MiniLM-L6-v2', device='cpu')
    
    # Smaller batch size for CPU
    batch_size = 16
    
    print("Computing embeddings for training data...")
    train_q1_embeddings = model.encode(
        train_df['question1'].tolist(),
        batch_size=batch_size,
        show_progress_bar=True,
        convert_to_numpy=True
    )
    train_q2_embeddings = model.encode(
        train_df['question2'].tolist(),
        batch_size=batch_size, 
        show_progress_bar=True,
        convert_to_numpy=True
    )
    
    print("Computing embeddings for test data...")
    test_q1_embeddings = model.encode(
        test_df['question1'].tolist(),
        batch_size=batch_size,
        show_progress_bar=True,
        convert_to_numpy=True
    )
    test_q2_embeddings = model.encode(
        test_df['question2'].tolist(),
        batch_size=batch_size,
        show_progress_bar=True,
        convert_to_numpy=True
    )
    
    print("Computing cosine similarities...")
    # Normalize and compute similarity
    train_q1_norm = train_q1_embeddings / np.linalg.norm(train_q1_embeddings, axis=1, keepdims=True)
    train_q2_norm = train_q2_embeddings / np.linalg.norm(train_q2_embeddings, axis=1, keepdims=True)
    test_q1_norm = test_q1_embeddings / np.linalg.norm(test_q1_embeddings, axis=1, keepdims=True)
    test_q2_norm = test_q2_embeddings / np.linalg.norm(test_q2_embeddings, axis=1, keepdims=True)
    
    train_sim = np.sum(train_q1_norm * train_q2_norm, axis=1)
    test_sim = np.sum(test_q1_norm * test_q2_norm, axis=1)
    
    train_df['embedding_cosine_similarity'] = train_sim
    test_df['embedding_cosine_similarity'] = test_sim
    
    print("CPU embedding computation completed successfully!")
    return train_df, test_df


# Usage:
train_df, test_df = compute_embedding_similarity(train_df, test_df, batch_size=32, chunk_size=10000)

In [ ]:
feature_cols=['lenq1','lenq2','len_diff','common_words','tfidf_cosine_sim','embedding_cosine_similarity']
X_train=train_df[feature_cols]
y_train=train_df['is_duplicate']
X_test=test_df[feature_cols]
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [ ]:
pip install xgboost

In [ ]:
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
# Check if XGBoost can use GPU
print(f"XGBoost version: {xgb.__version__}")

# Configure XGBoost for GPU
if torch.cuda.is_available():
    xgb_params = {
        'tree_method': 'gpu_hist',  # GPU acceleration
        'gpu_id': 0,
        'use_label_encoder': False,
        'eval_metric': 'logloss',
        'random_state': 42,
        'n_estimators': 100,
        'max_depth': 6,
        'learning_rate': 0.1
    }
    print("Using GPU-accelerated XGBoost")
else:
    xgb_params = {
        'use_label_encoder': False,
        'eval_metric': 'logloss',
        'random_state': 42,
        'n_estimators': 100,
        'max_depth': 6,
        'learning_rate': 0.1
    }
    print("Using CPU XGBoost")

# Train model
print("Training XGBoost model...")
xgb_model = xgb.XGBClassifier(**xgb_params)
cv_scores=cross_val_score(xgb_model,X_train,y_train,cv=5,scoring='accuracy')
f1_scores=cross_val_score(xgb_model,X_train,y_train,cv=5,scoring='f1')
roc_auc_scores=cross_val_score(xgb_model,X_train,y_train,cv=5,scoring='roc_auc')

print("Cross-validation accuracy scores:", cv_scores.mean())
print("Average F1 score:", f1_scores.mean())
print("Average ROC-AUC score:", roc_auc_scores.mean())

In [ ]:
#xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
xgb_model.fit(X_train, y_train)
y_test_probs=xgb_model.predict_proba(X_test)[:, 1]
y_test_pred=(y_test_probs>=0.5).astype(int)
#Prepare submission file
submission = pd.DataFrame({
    'id': test_df['id'],         
    'is_duplicate': y_test_pred
})
submission.to_csv('submission.csv', index=False)
print("Submission file created successfully!")